In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pathlib
import time
import os
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


import pandas as pd

from tqdm import tqdm



In [2]:
class NetflixOfficialCrwl:
    def __init__(self):
        self.chrome_options = Options()
        self.chrome_options.add_experimental_option("detach", True)
        self.chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
        self.service = Service(executable_path=ChromeDriverManager().install())

        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
        # self.chrome_options.add_argument(f'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36')

        self.path = pathlib.Path('../../data/netflix/netflix_title.csv')
        self.netflix_csv = pd.read_csv(self.path)

    def netflix_official_crwl(self,start_number=0):
        n_title = self.netflix_csv['nfid'][start_number:]
        driver = webdriver.Chrome(service=self.service,options=self.chrome_options)
        driver.implicitly_wait(10)
        result = pd.DataFrame()
        not_find = []

        for n_t in tqdm(n_title):
            url = 'https://www.netflix.com/kr/title/'
            url += str(n_t)
            driver.get(url)
            time.sleep(round(0.5+random.random(),2))
            try:
                title = driver.find_element(By.CSS_SELECTOR,'h1.title-title').text
                title = {'제목':title}
                try:
                    synopsis = driver.find_element(By.CSS_SELECTOR,'div.title-info-synopsis').text
                    synopsis = {'시놉시스':synopsis}
                except:
                    synopsis = {'시놉시스':0}

                genres = driver.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-genres')
                genres_str= ''
                for g in genres:
                    genres_str+=g.text
                genres_str = {'장르모음':genres_str}

                people = driver.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-cast')
                people_str= ''
                for p in people:
                    people_str+=p.text+','
                people_str = {'출연진':people_str}
                try:
                    feature = driver.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-mood-tag')
                    feature_str = ''
                    for f in feature:
                        feature_str+=f.text
                    feature_str = {'특징':feature_str}     
                except:
                    feature_str = {'특징':0}     

                metadata = driver.find_element(By.CSS_SELECTOR,'div.title-info-metadata-wrapper').text.split('|')
                if len(metadata)==3: metadata.insert(2,'0')
                    
                
                api_netflix = self.netflix_csv[self.netflix_csv['nfid']==n_t][['nfid','imdbid']].values[0]
                n_i_id = {'nfid':api_netflix[0],'imdbid':api_netflix[1]}

                df = pd.concat([pd.DataFrame([title]),pd.DataFrame([genres_str]),pd.DataFrame([synopsis]),pd.DataFrame([feature_str]),
                                pd.DataFrame([people_str]),pd.DataFrame([metadata],columns=['개봉일','등급','상영시간','기본장르']),pd.DataFrame([n_i_id])]
                                ,axis=1)
                result = pd.concat([result,df])
                # result.to_csv('test.csv',encoding='utf-8')
            except:
                # result.to_csv(f'n_{start_number}.csv',encoding='utf-8')
                print(n_t)
                not_find.append(n_t)
                continue
        result.to_csv('netflix_official_.csv',encoding='utf-8')
        pd.DataFrame(not_find).to_csv('not.find.csv',encoding='utf-8')
                
    

In [3]:
crwl = NetflixOfficialCrwl()
crwl.netflix_official_crwl()

  2%|▏         | 42/1833 [03:13<3:53:20,  7.82s/it]

70224562


 16%|█▋        | 299/1833 [25:27<3:21:16,  7.87s/it]

80211229


 19%|█▉        | 355/1833 [30:44<2:24:55,  5.88s/it]

...

 99%|█████████▉| 1812/1833 [2:33:17<02:05,  5.99s/it]

80231259


 99%|█████████▉| 1813/1833 [2:33:29<02:33,  7.67s/it]

70077556


100%|██████████| 1833/1833 [2:34:22<00:00,  5.05s/it]


In [6]:
test = pd.read_csv('netflix_official_.csv')

In [9]:
test

,제목,장르모음,시놉시스,특징,출연진,개봉일,등급,상영시간,기본장르,nfid,imdbid
0,우주 히어로 키드,"키즈 시리즈,가족이 함께 보는 시리즈,코미디 시리즈,만화 시리즈",지구를 지키는 영웅이 되고 싶어요. 이 꿈이 정말로 이뤄질 줄이야! 한 소년이 우주...,"유쾌 발랄,흥미진진","잭 피셔,톰 케니,어맨다 C. 밀러,킴 야브로,키스 퍼거슨,그레이 그리핀,릴리 로즈...",2021,\n전체관람가,시즌 3개,키즈,80244340,tt9248538
1,청소년 v 정부: 기후 정의를 외치다,"다큐멘터리 영화,자연 & 생태 다큐멘터리,과학 & 자연 다큐멘터리,사회 & 문화 다...",기후 위기를 초래한 미국 정부의 행태를 두고 볼 수 없었던 21명의 청소년 원고단....,"도발적인,감명을 주는",NaN,2020,\n15+,1시간 49분,다큐멘터리,81586492,tt13192640
2,하프 배드: 선 & 데빌,"로맨틱한 드라마,드라마,영국 작품,청소년 시리즈,액션 & 어드벤처 시리즈,판타지 시리즈",전쟁 중인 두 종족 사이에 운명적으로 끼게 된 소년. 끔찍한 학살을 일으킨 악명 높...,"폭력적인,흥미진진,로맨틱","제이 라이커고,나디아 파크스,에밀리앵 베크만스,폴 레디,이저벨 제스퍼 존스,캐런 코...",2022,\n청불,시즌 1개,판타지,81258637,tt16380676
3,Breaking Bad,"드라마,범죄 시리즈,미국 TV 프로그램,스릴러 시리즈","말기 암 진단이 그를 범죄의 세계로 밀어 넣었다. 아니, 그런 줄 알았다. 제자와 ...","폭력적인,어두운,긴장감 넘치는","브라이언 크랜스턴,에런 폴,안나 건,딘 노리스,벳시 브랜트,RJ 미티,밥 오든커크,...",2008,\n19+,시즌 5개,드라마 장르,70143836,tt0903747
4,우리의 지구,"가족이 함께 보는 시리즈,영국 작품,자연 & 생태 다큐멘터리,과학 & 자연 TV 프...","압도적인 스케일, 경이로운 영상미 그리고 전 인류를 향한 메시지. 우리 지구에 관한...",감명을 주는,"데이비드 애튼버러,",2019,\n전체관람가,시즌 2개,다큐멘터리,80049832,tt9253866
...,...,...,...,...,...,...,...,...,...,...,...
1801,해피 데스데이,"코미디 영화,호러 영화,미스터리 영화,미국 영화",숙취로 시작해 자신의 죽음으로 끝난 한 여성의 생일. 그런데 다음 날 그녀가 깨어나...,"무서운,긴장감 넘치는","제시카 로스,이즈리얼 브루사드,루비 모딘,찰스 에이킨,로라 클리프턴,제이슨 베일,롭...",2017,\n15+,1시간 36분,호러,80188939,tt5308322
1802,임금님의 사건수첩,"코미디 영화,스릴러 영화",흉흉한 소문이 돌고 괴이한 사건이 연이어 발생하는 조선. 이에 임금님이 직접 비밀 ...,긴장감 넘치는,"이선균,안재홍,김희원,주진모,장영남,경수진,정해인,김홍파,김응수,",2017,\n12+,1시간 54분,코미디,80188833,tt6817202
1803,오늘은 회사 쉬겠습니다,"일본 작품,코미디 시리즈,로맨틱 코미디 시리즈,일본 만화 원작 시리즈",서른 살 모태 솔로 하나에. 스물한 살의 연하 남친이 생겨 하늘을 날듯 행복하다. ...,로맨틱,"아야세 하루카,후쿠시 소타,타마키 히로시,나카 리이사,타구치 준노스케,지바 유다이,...",2014,\n15+,시즌 1개,로맨스,81410173,tt0120915
1804,알카티브,"사우디아라비아 작품,중동 영화,드라마 영화,미스터리 영화,사회 이슈 드라마 장르 영화",아름다운 인턴에게 마음을 온통 빼앗겨버린 회사원. 그녀를 쫓아 사막의 리조트로 향한...,긴장감 넘치는,"후삼 알하르티,림 알하비브,누르 알카드라,라하프 이브라힘,아심 알아와드,나디아 알모...",2023,\n15+,1시간 21분,드라마 장르,81354937,tt0119632


In [5]:
t = pd.read_csv('test.csv')

In [6]:
t

,Unnamed: 0,제목,장르모음,시놉시스,특징,출연진,개봉일,등급,상영시간,기본장르
0,0,우주 히어로 키드,"키즈 시리즈,가족이 함께 보는 시리즈,코미디 시리즈,만화 시리즈",지구를 지키는 영웅이 되고 싶어요. 이 꿈이 정말로 이뤄질 줄이야! 한 소년이 우주...,"유쾌 발랄,,흥미진진,","잭 피셔,톰 케니,어맨다 C. 밀러,킴 야브로,키스 퍼거슨,그레이 그리핀,릴리 로즈...",2021,\n전체관람가,시즌 3개,키즈
1,0,청소년 v 정부: 기후 정의를 외치다,"다큐멘터리 영화,자연 & 생태 다큐멘터리,과학 & 자연 다큐멘터리,사회 & 문화 다...",기후 위기를 초래한 미국 정부의 행태를 두고 볼 수 없었던 21명의 청소년 원고단....,"도발적인,,감명을 주는,",NaN,2020,\n15+,1시간 49분,다큐멘터리
2,0,하프 배드: 선 & 데빌,"로맨틱한 드라마,드라마,영국 작품,청소년 시리즈,액션 & 어드벤처 시리즈,판타지 시리즈",전쟁 중인 두 종족 사이에 운명적으로 끼게 된 소년. 끔찍한 학살을 일으킨 악명 높...,"폭력적인,,흥미진진,,로맨틱,","제이 라이커고,나디아 파크스,에밀리앵 베크만스,폴 레디,이저벨 제스퍼 존스,캐런 코...",2022,\n청불,시즌 1개,판타지
3,0,Breaking Bad,"드라마,범죄 시리즈,미국 TV 프로그램,스릴러 시리즈","말기 암 진단이 그를 범죄의 세계로 밀어 넣었다. 아니, 그런 줄 알았다. 제자와 ...","폭력적인,,어두운,,긴장감 넘치는,","브라이언 크랜스턴,에런 폴,안나 건,딘 노리스,벳시 브랜트,RJ 미티,밥 오든커크,...",2008,\n19+,시즌 5개,드라마 장르
4,0,우리의 지구,"가족이 함께 보는 시리즈,영국 작품,자연 & 생태 다큐멘터리,과학 & 자연 TV 프...","압도적인 스케일, 경이로운 영상미 그리고 전 인류를 향한 메시지. 우리 지구에 관한...","감명을 주는,","데이비드 애튼버러,",2019,\n전체관람가,시즌 2개,다큐멘터리
5,0,1899,"드라마,독일 작품,액션 & 어드벤처 시리즈,스릴러 시리즈","1899년, 이민자들을 싣고 뉴욕으로 향하던 배가 미스터리한 사건을 마주한다. 급격...","발상의 전환,,불길한,","에밀리 비첨,어나이린 바나드,안드레아스 피치만,미겔 베르나르데아우,조제 피멘탕,이저...",2022,\n청불,시즌 1개,스릴러


In [256]:
test['출연진'][0]

'잭 피셔 톰 케니 어맨다 C. 밀러 킴 야브로 키스 퍼거슨 그레이 그리핀 릴리 로즈 실버 프레드 타타시오르'

In [23]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")

In [24]:
browser = webdriver.Chrome(service=service,options=chrome_options)

browser.implicitly_wait(10)
browser.maximize_window()
url = f"https://www.netflix.com/kr/title/81586492"
browser.get(url)

    


In [28]:
t=browser.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-mood-tag')

In [29]:
for i in t:
    print(i.text)

도발적인,
감명을 주는


In [6]:
a=pd.read_csv('netflix_official_.csv',encoding='utf-8')

In [43]:
pd.read_csv('test.csv',encoding='utf-8')

,Unnamed: 0,제목,장르모음,시놉시스,특징,출연진,개봉일,등급,상영시간,기본장르,nfid,imdbid
0,0,우주 히어로 키드,"키즈 시리즈,가족이 함께 보는 시리즈,코미디 시리즈,만화 시리즈",지구를 지키는 영웅이 되고 싶어요. 이 꿈이 정말로 이뤄질 줄이야! 한 소년이 우주...,"유쾌 발랄,흥미진진","잭 피셔,톰 케니,어맨다 C. 밀러,킴 야브로,키스 퍼거슨,그레이 그리핀,릴리 로즈...",2021,\n전체관람가,시즌 3개,키즈,80244340,tt9248538
1,0,청소년 v 정부: 기후 정의를 외치다,"다큐멘터리 영화,자연 & 생태 다큐멘터리,과학 & 자연 다큐멘터리,사회 & 문화 다...",기후 위기를 초래한 미국 정부의 행태를 두고 볼 수 없었던 21명의 청소년 원고단....,"도발적인,감명을 주는",NaN,2020,\n15+,1시간 49분,다큐멘터리,81586492,tt13192640
2,0,하프 배드: 선 & 데빌,"로맨틱한 드라마,드라마,영국 작품,청소년 시리즈,액션 & 어드벤처 시리즈,판타지 시리즈",전쟁 중인 두 종족 사이에 운명적으로 끼게 된 소년. 끔찍한 학살을 일으킨 악명 높...,"폭력적인,흥미진진,로맨틱","제이 라이커고,나디아 파크스,에밀리앵 베크만스,폴 레디,이저벨 제스퍼 존스,캐런 코...",2022,\n청불,시즌 1개,판타지,81258637,tt16380676
3,0,Breaking Bad,"드라마,범죄 시리즈,미국 TV 프로그램,스릴러 시리즈","말기 암 진단이 그를 범죄의 세계로 밀어 넣었다. 아니, 그런 줄 알았다. 제자와 ...","폭력적인,어두운,긴장감 넘치는","브라이언 크랜스턴,에런 폴,안나 건,딘 노리스,벳시 브랜트,RJ 미티,밥 오든커크,...",2008,\n19+,시즌 5개,드라마 장르,70143836,tt0903747
4,0,우리의 지구,"가족이 함께 보는 시리즈,영국 작품,자연 & 생태 다큐멘터리,과학 & 자연 TV 프...","압도적인 스케일, 경이로운 영상미 그리고 전 인류를 향한 메시지. 우리 지구에 관한...",감명을 주는,"데이비드 애튼버러,",2019,\n전체관람가,시즌 2개,다큐멘터리,80049832,tt9253866
5,0,1899,"드라마,독일 작품,액션 & 어드벤처 시리즈,스릴러 시리즈","1899년, 이민자들을 싣고 뉴욕으로 향하던 배가 미스터리한 사건을 마주한다. 급격...","발상의 전환,불길한","에밀리 비첨,어나이린 바나드,안드레아스 피치만,미겔 베르나르데아우,조제 피멘탕,이저...",2022,\n청불,시즌 1개,스릴러,80214497,tt9319668
6,0,릭 앤 모티,"시트콤,SF 시리즈,코미디 시리즈,액션 & 어드벤처 시리즈,미국 TV 프로그램",술에 절어 사는 천재 과학자 릭은 투정이 심한 10대 손자 모티를 데리고 외계와 대...,"익살스러운,유쾌 발랄","저스틴 로일랜드,크리스 파넬,스펜서 그래머,세라 초크,",2013,\n청불,시즌 6개,코미디,80014749,tt2861424
7,0,마이클 조던: 더 라스트 댄스,"다큐시리즈,사회 & 문화 다큐멘터리,미국 TV 프로그램",농구 황제 마이클 조던과 시카고 불스를 다시 만난다. 1997-98 시즌의 미공개 ...,흥미진진,"마이클 조던,스코티 피펜,데니스 로드먼,",2020,\n15+,시즌 1개,다큐멘터리,80203144,tt8420184
8,0,데이비드 애튼버러: 우리의 지구를 위하여,"영국 작품,다큐멘터리 영화,자연 & 생태 다큐멘터리,과학 & 자연 다큐멘터리,실존 ...",바다가 죽어간다. 얼음이 사라진다. 인간이 파괴한 야생의 세계. 일생에 걸쳐 지구의...,거부할 수 없는 이끌림,"데이비드 애튼버러,",2020,\n전체관람가,1시간 23분,다큐멘터리,80216393,tt11989890
9,0,아바타 아앙의 전설,"키즈 시리즈,가족이 함께 보는 시리즈,만화 시리즈,판타지 시리즈","불의 제국이 일으킨 전쟁으로 균형이 깨져 버린 세상. 물, 불, 흙, 공기에 전부 ...",흥미진진,"자크 타일러,메이 휘트먼,잭 디세나,디 브래들리 베이커,단테 바스코,제시 플라워,마...",2005,\n전체관람가,시즌 3개,키즈,70142405,tt0417299


In [16]:
path = pathlib.Path('../../data/netflix/netflix_title.csv')
netflix_csv = pd.read_csv(path)
netflix_csv.head()

,Unnamed: 0,id,title,img,vtype,nfid,synopsis,avgrating,year,runtime,imdbid,poster,imdbrating,top250,top250tv,clist,titledate
0,0,68518,Kid Cosmic,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,series,80244340,A boy&#39;s superhero dreams come true when he...,0.00000,2021,0,tt9248538,https://m.media-amazon.com/images/M/MV5BNmEwNz...,10.0,0.0,0.0,"""CA"":""Canada"",""FR"":""France"",""DE"":""Germany"",""NL...",2021-02-03
1,1,79125,Youth v Gov,https://occ-0-2851-1432.1.nflxso.net/dnm/api/v...,movie,81586492,This documentary follows the journey of 21 you...,0.00000,2020,6554,tt13192640,https://m.media-amazon.com/images/M/MV5BM2ZjMz...,9.6,NaN,NaN,"""CA"":""Canada"",""FR"":""France"",""DE"":""Germany"",""NL...",2022-04-29
2,2,82867,The Bastard Son & The Devil Himself,https://occ-0-363-2567.1.nflxso.net/dnm/api/v6...,series,81258637,"Caught between two warring clans, the son of a...",0.00000,2022,0,tt16380676,https://m.media-amazon.com/images/M/MV5BNTRjYT...,9.6,NaN,NaN,"""CA"":""Canada"",""FR"":""France"",""DE"":""Germany"",""NL...",2022-10-28
3,3,26333,Breaking Bad,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,series,70143836,A terminally ill chemistry teacher teams with ...,4.48224,2008,0,tt0903747,https://images-na.ssl-images-amazon.com/images...,9.5,0.0,4.0,"""CA"":""Canada"",""FR"":""France"",""DE"":""Germany"",""NL...",2015-04-14
4,4,61645,Our Planet,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,series,80049832,Experience our planet&#39;s natural beauty and...,0.00000,2019,0,tt9253866,https://m.media-amazon.com/images/M/MV5BMDE5YT...,9.3,0.0,8.0,"""CA"":""Canada"",""FR"":""France"",""DE"":""Germany"",""NL...",2019-04-05


In [37]:
netflix_csv[netflix_csv['nfid']==81586492][['nfid','imdbid']].values[0][0]

81586492